In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pickle

def train_and_predict_catboost(train_filepath, valid_filepath, test_filepath, model_output_path, submission_output_path):
    """
    Обучает CatBoostClassifier, делает предсказания на тестовых данных и сохраняет результаты в файл submission_catboost.csv.

    Args:
        train_filepath (str): Путь к CSV файлу с обучающими данными.
        valid_filepath (str): Путь к CSV файлу с валидационными данными.
        test_filepath (str): Путь к CSV файлу с тестовыми данными.
        model_output_path (str): Путь к файлу, куда нужно сохранить обученную модель.
        submission_output_path (str): Путь к файлу, куда нужно сохранить предсказания.
    """

    try:
        train_df = pd.read_csv(train_filepath)
        valid_df = pd.read_csv(valid_filepath)
        test_df = pd.read_csv(test_filepath)
    except FileNotFoundError:
        print(f"Ошибка: Один из файлов не найден.")
        return
    except Exception as e:
      print(f"Ошибка при чтении файла: {e}")
      return

    # Разделяем данные на признаки (X) и целевую переменную (y)
    X_train = train_df.drop(columns=['target'])
    y_train = train_df['target']
    X_valid = valid_df.drop(columns=['target'])
    y_valid = valid_df['target']
    X_test = test_df.drop(columns=['target'], errors='ignore') # убираем таргет, если он есть

    # Определяем категориальные признаки
    categorical_features_indices = [i for i, col in enumerate(X_train.columns) if X_train[col].dtype == 'object']


    # Создание CatBoost Pool для оптимальной работы с категориальными данными
    train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)
    valid_pool = Pool(data=X_valid, label=y_valid, cat_features=categorical_features_indices)
    test_pool = Pool(data=X_test, cat_features=categorical_features_indices)


    # Инициализация и обучение CatBoostClassifier
    model = CatBoostClassifier(
        iterations=200,
        learning_rate=0.1,
        depth=6,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=42,
        verbose=10
    )
    model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=20)

    # Предсказание на валидационных данных
    y_pred_valid = model.predict(valid_pool)
    y_prob_valid = model.predict_proba(valid_pool)[:, 1]

    # Оценка модели на валидационных данных
    accuracy = accuracy_score(y_valid, y_pred_valid)
    precision = precision_score(y_valid, y_pred_valid)
    recall = recall_score(y_valid, y_pred_valid)
    f1 = f1_score(y_valid, y_pred_valid)
    auc = roc_auc_score(y_valid, y_prob_valid)

    # Вывод метрик оценки на валидационных данных
    print(f"\nМетрики оценки на валидационных данных:")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-score:  {f1:.4f}")
    print(f"  AUC:       {auc:.4f}")

    # Предсказание на тестовых данных
    test_pred_prob = model.predict_proba(test_pool)[:, 1]
    test_ids = test_df['id'] if 'id' in test_df.columns else test_df.index

    # Создание DataFrame для сохранения предсказаний
    submission_df = pd.DataFrame({'id': test_ids, 'target': test_pred_prob})

    # Сохранение предсказаний в файл submission_catboost.csv
    submission_df.to_csv(submission_output_path, index=False)
    print(f"\nПредсказания на тестовых данных сохранены в файл: {submission_output_path}")


    # Сохранение модели
    try:
        with open(model_output_path, 'wb') as f:
            pickle.dump(model, f)
        print(f"\nМодель сохранена в файл: {model_output_path}")
    except Exception as e:
        print(f"Ошибка при сохранении модели: {e}")

# Пути к файлам
train_file_path = "cleaned_data_filled_cleaned2.csv"
valid_file_path = "cleaned_data_filled_cleaned1.csv"
test_file_path = "test.csv" # Замените на свой путь к файлу с тестовыми данными
model_path = "catboost_model.pkl"
submission_path = "submission_catboost.csv"


# Вызов функции для обучения модели и предсказания
train_and_predict_catboost(train_file_path, valid_file_path, test_file_path, model_path, submission_path)

ModuleNotFoundError: No module named 'catboost'